# Working with Git in VSCode

### Introduction

In previous lessons we moved through the steps of changing our code to complete a task.  However, we did not really address a git workflow.  In this lesson, we'll move through some of the steps we should move through in git when making a commit.

### Getting started

1. Get oriented

We can use `git status` to see where we are.

`git status`

> <img src="./langchain-commit.png" width="60%">

> Here we see three files changes.  But you may not have any changes to the Makefile.  That's fine/good.

Currently, we may be on the `master` branch, which is equivalent to the `main` branch in many other codebases.  Now we probably should have made a new branch when we first started writing some code.  But don't worry, we can create a new branch now.

* `git checkout -b standardize_perplexity_params`

This creates a new branch, and moves the changes over to the that branch `standardize_perplexity_params`.  Then let's check where we are with `git status`.

* `git status`
```
On branch standardize_perplexity_params
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
        modified:   libs/community/Makefile
        modified:   libs/community/langchain_community/chat_models/perplexity.py
        modified:   libs/community/tests/unit_tests/chat_models/test_perplexity.py
```

From there, we begin to make a commit, but before we do we should look through all of the changes we have made so far.  This is where we can use `git diff`.

Oftentimes, it's easier to view those changes file by file.  Let's start with the `Makefile`.

* `git diff Makefile`

<img src="./diff-display.png" width="60%">

> **Warning**: This places us in `vim`.  To exit, just type the letter `q`.  Or `ESC`, `:`, `q`.

Ok, so here, it looks like I accidentally deleted a space.  A git commit is an authorship of our changes.  Each component should go together.  This is just a mistake, and we should undo the change.  The easiest way to undo it is with `git checkout` like so:

* `git checkout libs/community/Makefile`

This will undo our changes from the Makefile.  We can see this if we type `git status` again.

```bash
git status
On branch standardize_perplexity_params
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
        modified:   libs/community/langchain_community/chat_models/perplexity.py
        modified:   libs/community/tests/unit_tests/chat_models/test_perplexity.py
```

> So as you can see, `git checkout` can be used a couple different ways.  We can use it to switch branches.  Or we can also use it to remove changes made to one or more files.

Ok, so now it looks like we are a bit closer to only the changes we prefer.  But we should make sure.  Once again call `git diff` on each of the files.

> You can press the `space` key to move down through the file, and press `q` to exit the file.

* `git diff libs/community/langchain_community/chat_models/perplexity.py`
* `git diff libs/community/tests/unit_tests/chat_models/test_perplexity.py`

If there are any edits to make, simply edit the file.  But my changes look pretty good.

### Formatting code

Now if you look at the [contributing code](https://python.langchain.com/docs/contributing/code/) section, they tell us that after running our tests (which we did earlier), we should also properly format the code.

This [contributing code](https://python.langchain.com/docs/contributing/code/) section tells us how to do so.

> <img src="./make-lint.png" width="70%">

In additon, we can check the Makefile to see what `lint_diff` is doing.

```yaml
lint lint_diff lint_package lint_tests:
	./scripts/check_pydantic.sh .
	./scripts/lint_imports.sh
	poetry run ruff .
	[ "$(PYTHON_FILES)" = "" ] || poetry run ruff format $(PYTHON_FILES) --diff
```

So this is using the ruff library, which is a formatting library.  We can always ask Chatgpt to explain further.  But let's keep going.

Let's run:

```bash
make lint_diff
```



For me, this did not actually work.  So in addition, we can also run:

```bash
poetry run ruff libs/community/tests/unit_tests/chat_models/test_perplexity.py
libs/community/tests/unit_tests/chat_models/test_perplexity.py:33:89: E501 Line too long (96 > 88)
Found 1 error.
```

This indicates that one of my lines is too long.  But ruff will fix it for me, if I run `poetry run ruff format`.  Let's do that on each of our changed files.

```bash
poetry run ruff format libs/community/tests/unit_tests/chat_models/test_perplexity.py

poetry run ruff format libs/community/langchain_community/chat_models/perplexity.py
```

If you look at the files now, you should see your code properly formatted.  We are now almost ready to to make a commit, but it's best to run our tests again.

```bash
make tests
```

### Finally a commit

Ok, so if all of our tests pass, then we can make a commit.

```bash
git status 
#check to make sure only the files we want have been changed

git add -A
# Add changes to staging
```

Then confirm we did this correctly.

```bash
git status
On branch standardize_perplexity_params
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
        modified:   libs/community/langchain_community/chat_models/perplexity.py
        modified:   libs/community/tests/unit_tests/chat_models/test_perplexity.py
```

* Making a multiline commit

When making a commit, it's a good idea to have a description of what we are contributing as well as a link to the issue or feature request.  For example, this is fairly good.

```markdown
community:perplexity[patch]: standardize init args

updated pplx_api_key and request_timeout so that aliased to api_key, and timeout respectively. Added test that both continue to set the same underlying attributes.

Related to [20085](https://github.com/langchain-ai/langchain/issues/20085)
```

But how do we have a multiline commit?  Ideally, we can use `git commit -v`, where v stands for visual.  But unfortunately, this did not work for me.  

So instead we can write our message in a separate file, and then specify that the contents of the file should be our commit message.  To accomplish this, create a new file in the root of the project directory.  

```
touch sample-message.txt
```

And then we can copy and paste the message above into the text file.  And then we can make our commit, where the message is in the file.

* `git commit -F sample-message.txt`

If you type `git log -2`, you should now see your multiline commit.

```bash
Author: Jeff K <someemail@gmail.com>
Date:   Wed May 1 19:05:39 2024 +0000

    community:perplexity[patch]: standardize init args
    
    updated pplx_api_key and request_timeout so that aliased to api_key, and timeout respectively. Added test that both continue to set the same underlying attributes.
    
    Related to [20085](https://github.com/langchain-ai/langchain/issues/20085)
```

That looks pretty good.  

* Adding an author

So above, I have a nice byline to both have some credit (and some blame) for my commit.  If there's an issue, someone can email me, or ask me about my commit.  If you don't see your own name on your commit, you can `amend` your commit with the following:

```bash
git commit --amend --author="Author Name <email@example.com>" -F sample-message.txt
```

> And just replace the author name and email as appropriate.

Now we do have one more key step which is to rebase our branch to ensure it is up to date with the current version of the codebase.  But we'll cover that in the next lesson.

### Summary

In this lesson, we covered various details of a good workflow.  These include the following:

1. Using `git status` and `git diff` to see what changes have been made to the codebase.  

And then being careful to only add those changes that we want.  We can use `git diff file_name.py` to check the changes made to one file at a time.

2. Running the specified linter.  

We used `poetry run ruff file_name.py` to spot any error, and `poetry run ruff format file_name.py` to actually change the file.

3. Adding a multi-line commit that links to the issue we are fixing

This is key.  We want our commit to link to the broader issue that it is trying to fix.  We can create a new file, eg. `sample-message.txt`) and then running `git commit -F sample-message.txt`.

4. Signing off

Finally, we can add our name and email

`git commit --amend --author="Author Name <email@example.com>" -F sample-message.txt`